# <span style="font-width:bold; font-size: 3rem; color:#1EB182;"><img src="images/icon102.png" width="38px"></img> **Hopsworks Feature Store** </span><span style="font-width:bold; font-size: 3rem; color:#333;">- Part 03: Training Data & Feature views</span>

<span style="font-width:bold; font-size: 1.4rem;">This is the second part of the quick start series of tutorials about Hopsworks Feature Store. This notebook explains how to read from a feature group and create training dataset within the feature store</span>

## 🗒️ In this notebook we will see how to create a training dataset from the feature groups: 

1. Retrieving Feature Groups.
2. Defining Transformation functions.
4. Feature View creation.
5. Training Dataset with training, validation and test data.

![part2](images/02_training-dataset.png) 

## <span style="color:#ff5f27;"> 🔮 Connecting to Hopsworks Feature Store </span>

In [1]:
import hopsworks

project = hopsworks.login()

fs = project.get_feature_store() 

Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/167
Connected. Call `.close()` to terminate connection gracefully.


## <span style="color:#ff5f27;">🪄 Retrieving Feature Groups</span>

In [2]:
air_quality_fg = fs.get_or_create_feature_group(
    name = 'air_quality_fg',
    version = 1
)
weather_fg = fs.get_or_create_feature_group(
    name = 'weather_fg',
    version = 1
)

## <span style="color:#ff5f27;">💼 Query Preparation</span>

In [3]:
query = air_quality_fg.select_all().join(weather_fg.select_all())

query_show = query.show(5)
col_names = query_show.columns

query_show

2022-09-12 15:21:44,470 INFO: USE `maksym00_featurestore`
2022-09-12 15:21:45,676 INFO: WITH right_fg0 AS (SELECT *
FROM (SELECT `fg1`.`city` `city`, `fg1`.`aqi` `aqi`, `fg1`.`date` `date`, `fg1`.`iaqi_h` `iaqi_h`, `fg1`.`iaqi_p` `iaqi_p`, `fg1`.`iaqi_pm10` `iaqi_pm10`, `fg1`.`iaqi_t` `iaqi_t`, `fg1`.`o3_avg` `o3_avg`, `fg1`.`o3_max` `o3_max`, `fg1`.`o3_min` `o3_min`, `fg1`.`pm10_avg` `pm10_avg`, `fg1`.`pm10_max` `pm10_max`, `fg1`.`pm10_min` `pm10_min`, `fg1`.`pm25_avg` `pm25_avg`, `fg1`.`pm25_max` `pm25_max`, `fg1`.`pm25_min` `pm25_min`, `fg1`.`uvi_avg` `uvi_avg`, `fg1`.`uvi_max` `uvi_max`, `fg1`.`uvi_min` `uvi_min`, `fg1`.`city` `join_pk_city`, `fg1`.`date` `join_pk_date`, `fg1`.`date` `join_evt_date`, `fg0`.`tempmax` `tempmax`, `fg0`.`tempmin` `tempmin`, `fg0`.`temp` `temp`, `fg0`.`feelslikemax` `feelslikemax`, `fg0`.`feelslikemin` `feelslikemin`, `fg0`.`feelslike` `feelslike`, `fg0`.`dew` `dew`, `fg0`.`humidity` `humidity`, `fg0`.`precip` `precip`, `fg0`.`precipprob` `precipprob`, 

,city,aqi,date,iaqi_h,iaqi_p,iaqi_pm10,iaqi_t,o3_avg,o3_max,o3_min,...,windgust,windspeed,winddir,pressure,cloudcover,visibility,solarradiation,solarenergy,uvindex,conditions
0,Kyiv,2,1662670800000,99.90,1021.3,1,10.00,20,33,6,...,36.4,14.8,87.8,1022.5,71.8,24.1,146.3,12.6,5.0,"Rain, Partially cloudy"
1,Kyiv,4,1662584400000,99.90,1017.6,2,12.45,22,32,10,...,24.8,9.7,132.9,1019.4,48.4,24.1,217.9,18.7,7.0,Partially cloudy
2,Kyiv,2,1662498000000,65.58,1020.0,1,21.02,22,32,10,...,24.5,9.7,267.0,1022.3,34.8,24.1,227.5,19.6,7.0,Partially cloudy
3,Kyiv,6,1662411600000,98.78,1022.7,2,13.91,19,27,11,...,24.5,9.7,267.0,1022.3,34.8,24.1,227.5,19.6,7.0,Partially cloudy
4,Malmo,23,1662670800000,86.00,1006.3,7,12.70,33,35,32,...,46.1,28.0,127.0,1010.3,87.1,17.1,171.3,14.8,6.0,"Rain, Partially cloudy"


---

## <span style="color:#ff5f27;">🧑🏻‍🔬 Transformation functions</span>

Hopsworks Feature Store provides functionality to attach transformation functions to training datasets.

Hopsworks Feature Store also comes with built-in transformation functions such as `min_max_scaler`, `standard_scaler`, `robust_scaler` and `label_encoder`.

In [4]:
[t_func.name for t_func in fs.get_transformation_functions()]

['min_max_scaler', 'robust_scaler', 'standard_scaler', 'label_encoder']

We can retrieve transformation function we need .

To attach transformation function to training dataset provide transformation functions as dict, where key is feature name and value is online transformation function name.

Also training dataset must be created from the Query object. Once attached transformation function will be applied on whenever save, insert and get_serving_vector methods are called on training dataset object.

In [5]:
# Load transformation functions.
standard_scaler = fs.get_transformation_function(name = 'standard_scaler')
label_encoder = fs.get_transformation_function(name = 'label_encoder')

In [6]:
category_cols = ['city','date','conditions']

mapping_transformers = {col_name:standard_scaler for col_name in col_names if col_name not in category_cols}
category_cols = {col_name:label_encoder for col_name in category_cols if col_name != 'date'}

mapping_transformers.update(category_cols)

---

## <span style="color:#ff5f27;"> ⚙️ Feature View Creation </span>

`Feature Views` stands between **Feature Groups** and **Training Dataset**. Сombining **Feature Groups** we can create **Feature Views** which store a metadata of our data. Having **Feature Views** we can create **Training Dataset**.

The Feature Views allows schema in form of a query with filters, define a model target feature/label and additional transformation functions.

In order to create Feature View we can use `FeatureStore.create_feature_view()` method.

We can specify next parameters:

- `name` - name of a feature group.

- `version` - version of a feature group.

- `labels`- our target variable.

- `transformation_functions` - functions to transform our features.

- `query` - query object with data.

In [7]:
feature_view = fs.create_feature_view(
    name = 'air_quality_fv',
    version = 1,
    transformation_functions = mapping_transformers,
    query = query
)

Feature view created successfully, explore it at 
https://c.app.hopsworks.ai:443/p/167/fs/109/fv/air_quality_fv/version/1


For now `Feature View` is saved in Hopsworks and we can retrieve it using `FeatureStore.get_feature_view()`.

In [8]:
feature_view = fs.get_feature_view(
    name = 'air_quality_fv',
    version = 1
)

---

## <span style="color:#ff5f27;"> 🏋️ Training Dataset Creation</span>

In Hopsworks training data is a query where the projection (set of features) is determined by the parent FeatureView with an optional snapshot on disk of the data returned by the query.

**Training Dataset  may contain splits such as:** 
* Training set - the subset of training data used to train a model.
* Validation set - the subset of training data used to evaluate hparams when training a model
* Test set - the holdout subset of training data used to evaluate a mode

To create training dataset we use `FeatureView.create_training_data()` method.

Here are some importand things:

- It will inherit the name of FeatureView.

- The feature store currently supports the following data formats for
training datasets: **tfrecord, csv, tsv, parquet, avro, orc**.

- We can choose necessary format using **data_format** parameter.

- **start_time** and **end_time** in order to filter dataset in specific time range.

#### <span style="color:#ff5f27;"> ⛳️ Dataset with train, validation and test splits</span>

In [9]:
feature_view.create_training_data()

Training dataset job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/167/jobs/named/air_quality_fv_1_1_create_fv_td_12092022122417/executions


(1, <hsfs.core.job.Job at 0x7fc9469a46d0>)

#### <span style="color:#ff5f27;"> Next Steps</span>

In the next notebook, we will train a model on the Training Dataset we created in this notebook.